<a href="https://colab.research.google.com/github/nhgn1711/insurtech-assets-111/blob/main/%C4%91%E1%BA%A7y_%C4%91%E1%BB%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import:

In [3]:
!pip install yfinance tensorflow plotly pandas numpy scikit-learn statsmodels keras-tuner ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=669dfc79b6ca4f8e7b3af30f7d1148a9d9bea9c9fce062f054af6a78efa9a135
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [4]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU, Layer, Attention, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from google.colab import drive
import warnings
from statsmodels.tsa.arima.model import ARIMA
from keras_tuner import RandomSearch
import ta  # Technical Analysis library
warnings.filterwarnings('ignore')

# Mount Google Drive
drive.mount('/content/drive')
MODEL_PATH = '/content/drive/My Drive/lstm_model_optimized.h5'

Mounted at /content/drive


## Hàm:

In [5]:
# ============= CẢI TIẾN 1: FEATURE ENGINEERING NÂNG CAO =============
def add_advanced_indicators(df, vnindex_df):
    """Thêm nhiều chỉ báo kỹ thuật nâng cao"""
    # Moving Averages
    df['MA7'] = df['close'].rolling(window=7).mean()
    df['MA20'] = df['close'].rolling(window=20).mean()
    df['MA50'] = df['close'].rolling(window=50).mean()
    df['MA200'] = df['close'].rolling(window=200).mean()

    # Exponential Moving Averages
    df['EMA12'] = df['close'].ewm(span=12).mean()
    df['EMA26'] = df['close'].ewm(span=26).mean()

    # RSI (Relative Strength Index)
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()

    # MACD
    macd = ta.trend.MACD(df['close'])
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()

    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df['close'], window=20)
    df['BB_high'] = bollinger.bollinger_hband()
    df['BB_low'] = bollinger.bollinger_lband()
    df['BB_mid'] = bollinger.bollinger_mavg()
    df['BB_width'] = (df['BB_high'] - df['BB_low']) / df['BB_mid']

    # ATR (Average True Range) - Volatility
    df['ATR'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()

    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
    df['Stoch_K'] = stoch.stoch()
    df['Stoch_D'] = stoch.stoch_signal()

    # Volume indicators
    df['Volume_MA10'] = df['volume'].rolling(window=10).mean()
    df['Volume_MA20'] = df['volume'].rolling(window=20).mean()
    df['Volume_ratio'] = df['volume'] / df['Volume_MA20']

    # Price changes
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    df['volatility'] = df['returns'].rolling(window=20).std()

    # Lagged features
    for lag in [1, 2, 3, 5, 7]:
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'returns_lag_{lag}'] = df['returns'].shift(lag)

    # Price momentum
    df['momentum_5'] = df['close'] / df['close'].shift(5) - 1
    df['momentum_10'] = df['close'] / df['close'].shift(10) - 1

    # VN Index - try multiple symbols
    if not vnindex_df.empty:
        vnindex_df['VNI_returns'] = vnindex_df['close'].pct_change()
        vnindex_df['VNI_MA20'] = vnindex_df['close'].rolling(window=20).mean()
        df = df.merge(
            vnindex_df[['date', 'close', 'VNI_returns', 'VNI_MA20']].rename(
                columns={'close': 'VNINDEX'}
            ),
            on='date',
            how='left'
        )
        df[['VNINDEX', 'VNI_returns', 'VNI_MA20']] = df[['VNINDEX', 'VNI_returns', 'VNI_MA20']].fillna(method='ffill').fillna(method='bfill')
        print("✓ Đã thêm VN Index vào features")
    else:
        print("ℹ VN Index không khả dụng, tiếp tục với các features khác")

    # Day of week effect
    df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek

    # Forward fill and drop NaN
    df = df.fillna(method='ffill').fillna(method='bfill')
    df = df.dropna()

    return df

# ============= CẢI TIẾN 2: MÔ HÌNH LSTM VỚI ATTENTION =============
class AttentionLayer(Layer):
    """Custom Attention Layer"""
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], input_shape[-1]),
                                initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(input_shape[-1],),
                                initializer='zeros', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        a = tf.nn.softmax(e, axis=1)
        output = x * a
        return tf.reduce_sum(output, axis=1)

def build_advanced_lstm_model(hp, n_features, sequence_length):
    """Mô hình LSTM nâng cao với Bidirectional và Attention"""
    inputs = Input(shape=(sequence_length, n_features))

    # Bidirectional LSTM layers
    x = Bidirectional(LSTM(
        units=hp.Int('units_1', min_value=64, max_value=256, step=32),
        return_sequences=True,
        activation='tanh'
    ))(inputs)
    x = Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.4, step=0.1))(x)

    x = Bidirectional(LSTM(
        units=hp.Int('units_2', min_value=32, max_value=128, step=32),
        return_sequences=True
    ))(x)
    x = Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.4, step=0.1))(x)

    # Attention mechanism
    x = AttentionLayer()(x)

    # Dense layers
    x = Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
        loss='huber',  # Robust to outliers
        metrics=['mae', 'mse']
    )
    return model

# ============= CẢI TIẾN 3: GBM MONTE CARLO SIMULATION =============
def geometric_brownian_motion(df, n_days, n_sim=1000):
    """Monte Carlo với Geometric Brownian Motion"""
    returns = df['close'].pct_change().dropna()
    mu = returns.mean()
    sigma = returns.std()
    last_price = df['close'].iloc[-1]

    dt = 1/252  # Trading days
    simulations = np.zeros((n_days, n_sim))

    for i in range(n_sim):
        prices = [last_price]
        for _ in range(n_days - 1):
            shock = np.random.normal(mu * dt, sigma * np.sqrt(dt))
            price = prices[-1] * np.exp(shock)
            prices.append(price)
        simulations[:, i] = prices

    mean_pred = np.mean(simulations, axis=1)
    percentile_5 = np.percentile(simulations, 5, axis=1)
    percentile_95 = np.percentile(simulations, 95, axis=1)

    return mean_pred, percentile_5, percentile_95, simulations

# ============= CẢI TIẾN 4: MEAN-REVERTING MODEL (ORNSTEIN-UHLENBECK) =============
def mean_reverting_forecast(df, n_days, n_sim=1000):
    """Ornstein-Uhlenbeck process for mean-reverting stocks"""
    prices = df['close'].values
    log_prices = np.log(prices)

    # Estimate parameters
    theta = 0.1  # Speed of mean reversion
    mu = np.mean(log_prices[-60:])  # Long-term mean
    sigma = np.std(np.diff(log_prices[-60:]))

    last_log_price = log_prices[-1]
    dt = 1/252

    simulations = np.zeros((n_days, n_sim))

    for i in range(n_sim):
        log_price = last_log_price
        path = [np.exp(log_price)]
        for _ in range(n_days - 1):
            dW = np.random.normal(0, np.sqrt(dt))
            log_price = log_price + theta * (mu - log_price) * dt + sigma * dW
            path.append(np.exp(log_price))
        simulations[:, i] = path

    mean_pred = np.mean(simulations, axis=1)
    return mean_pred

# ============= CẢI TIẾN 5: ENSEMBLE MODEL =============
def ensemble_forecast(lstm_pred, gbm_pred, mr_pred, arima_pred, weights=None):
    """Kết hợp các mô hình với trọng số tối ưu"""
    if weights is None:
        # Default weights (có thể điều chỉnh dựa trên validation performance)
        weights = [0.4, 0.3, 0.2, 0.1]  # LSTM, GBM, Mean-Reverting, ARIMA

    ensemble = (weights[0] * lstm_pred +
                weights[1] * gbm_pred +
                weights[2] * mr_pred +
                weights[3] * arima_pred)
    return ensemble

# ============= CẢI TIẾN 6: RISK METRICS =============
def calculate_risk_metrics(simulations, current_price, confidence_level=0.95):
    """Tính toán các chỉ số rủi ro"""
    returns = (simulations[-1] - current_price) / current_price

    # Value at Risk (VaR)
    var = np.percentile(returns, (1 - confidence_level) * 100)

    # Conditional VaR (Expected Shortfall)
    cvar = returns[returns <= var].mean()

    # Maximum Drawdown
    cummax = np.maximum.accumulate(simulations, axis=0)
    drawdown = (simulations - cummax) / cummax
    max_drawdown = np.min(drawdown)

    # Probability of profit
    prob_profit = (returns > 0).sum() / len(returns)

    return {
        'VaR_95': var,
        'CVaR_95': cvar,
        'Max_Drawdown': max_drawdown,
        'Prob_Profit': prob_profit
    }

# ============= EVALUATION METRICS =============
def calculate_metrics(y_true, y_pred):
    """Tính toán đầy đủ các metrics"""
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)

    # Directional Accuracy
    direction_true = np.diff(y_true) > 0
    direction_pred = np.diff(y_pred) > 0
    dir_accuracy = np.mean(direction_true == direction_pred) * 100

    return {
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'Directional_Accuracy': dir_accuracy
    }

# ============= PREPARE DATA =============
SEQUENCE_LENGTH = 60

def prepare_data_advanced(df, sequence_length=SEQUENCE_LENGTH):
    """Chuẩn bị dữ liệu với nhiều features"""
    # Select important features
    feature_cols = [
        'close', 'MA20', 'MA50', 'RSI', 'MACD', 'MACD_signal',
        'BB_width', 'ATR', 'Volume_ratio', 'returns', 'volatility',
        'momentum_5', 'Stoch_K'
    ]

    # Add VN Index if available
    if 'VNINDEX' in df.columns:
        feature_cols.extend(['VNINDEX', 'VNI_returns'])

    features = df[feature_cols].values

    # Use RobustScaler for better handling of outliers
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(features)

    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i])
        y.append(scaled_data[i, 0])  # Predict close

    return np.array(X), np.array(y), scaler

# ============= TRAINING WITH WALK-FORWARD VALIDATION =============
def train_with_walk_forward(X, y, n_splits=5):
    """Train với Time Series Cross-Validation"""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    scores = []

    for train_idx, val_idx in tscv.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Train model (simplified for demo)
        # In production, would train full model here

        scores.append({'train_size': len(X_train), 'val_size': len(X_val)})

    return scores

# ============= MAIN TRAINING FUNCTION =============
def train_advanced_model(X, y, epochs=100, batch_size=32):
    """Train mô hình với các cải tiến"""
    n_features = X.shape[2]
    sequence_length = X.shape[1]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    # Hyperparameter tuning
    tuner = RandomSearch(
        lambda hp: build_advanced_lstm_model(hp, n_features, sequence_length),
        objective='val_loss',
        max_trials=10,
        executions_per_trial=1,
        directory='tuner_dir_advanced',
        project_name='lstm_advanced'
    )

    print("🔍 Đang tìm kiếm siêu tham số tối ưu...")
    tuner.search(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )

    best_model = tuner.get_best_models(num_models=1)[0]

    # Evaluate
    val_pred = best_model.predict(X_val, verbose=0)
    metrics = calculate_metrics(y_val, val_pred.flatten())

    print(f"✓ LSTM Validation Metrics:")
    print(f"  MAE: {metrics['MAE']:.6f} | RMSE: {metrics['RMSE']:.6f}")
    print(f"  MAPE: {metrics['MAPE']:.2f}% | R²: {metrics['R2']:.4f}")
    print(f"  Directional Accuracy: {metrics['Directional_Accuracy']:.2f}%")

    return best_model, metrics

# ============= FETCH DATA =============
def fetch_stock_data(symbol, start_date, end_date, retries=3):
    """Fetch stock data with retry logic"""
    for attempt in range(retries):
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(start=start_date, end=end_date, interval='1d')
            if df.empty:
                raise ValueError("Không có dữ liệu")
            df = df.reset_index()
            df['date'] = pd.to_datetime(df['Date']).dt.date
            df = df[['date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
            print(f"✓ Lấy được {len(df)} ngày dữ liệu cho {symbol}")
            return df
        except Exception as e:
            if attempt < retries - 1:
                print(f"⚠ Thử lại {attempt + 1}/{retries}...")
                time.sleep(2 ** attempt)
            else:
                print(f"✗ Không thể lấy dữ liệu {symbol}: {e}")
    return pd.DataFrame()

### Phân tích mô tả công ty:

In [6]:
# ============= COMPANY FUNDAMENTAL ANALYSIS =============
def get_company_fundamentals(symbol):
    """Lấy thông tin tài chính cơ bản của công ty"""
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info

        fundamentals = {
            # Thông tin công ty
            'company_name': info.get('longName', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'industry': info.get('industry', 'N/A'),
            'website': info.get('website', 'N/A'),
            'employees': info.get('fullTimeEmployees', 'N/A'),

            # Giá và vốn hóa
            'current_price': info.get('currentPrice', info.get('regularMarketPrice', 'N/A')),
            'market_cap': info.get('marketCap', 'N/A'),
            'enterprise_value': info.get('enterpriseValue', 'N/A'),

            # Chỉ số định giá
            'pe_ratio': info.get('trailingPE', info.get('forwardPE', 'N/A')),
            'pb_ratio': info.get('priceToBook', 'N/A'),
            'ps_ratio': info.get('priceToSalesTrailing12Months', 'N/A'),
            'peg_ratio': info.get('pegRatio', 'N/A'),

            # Chỉ số sinh lời
            'profit_margin': info.get('profitMargins', 'N/A'),
            'operating_margin': info.get('operatingMargins', 'N/A'),
            'roe': info.get('returnOnEquity', 'N/A'),
            'roa': info.get('returnOnAssets', 'N/A'),

            # Tăng trưởng
            'revenue_growth': info.get('revenueGrowth', 'N/A'),
            'earnings_growth': info.get('earningsGrowth', 'N/A'),

            # Cổ tức
            'dividend_yield': info.get('dividendYield', 'N/A'),
            'dividend_rate': info.get('dividendRate', 'N/A'),
            'payout_ratio': info.get('payoutRatio', 'N/A'),

            # Tình hình tài chính
            'total_cash': info.get('totalCash', 'N/A'),
            'total_debt': info.get('totalDebt', 'N/A'),
            'current_ratio': info.get('currentRatio', 'N/A'),
            'quick_ratio': info.get('quickRatio', 'N/A'),
            'debt_to_equity': info.get('debtToEquity', 'N/A'),

            # Hiệu suất
            'beta': info.get('beta', 'N/A'),
            '52w_high': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52w_low': info.get('fiftyTwoWeekLow', 'N/A'),
            '50d_avg': info.get('fiftyDayAverage', 'N/A'),
            '200d_avg': info.get('twoHundredDayAverage', 'N/A'),

            # Khối lượng
            'avg_volume': info.get('averageVolume', 'N/A'),
            'avg_volume_10d': info.get('averageVolume10days', 'N/A'),

            # Khuyến nghị
            'recommendation': info.get('recommendationKey', 'N/A'),
            'target_price': info.get('targetMeanPrice', 'N/A'),
        }

        return fundamentals
    except Exception as e:
        print(f"⚠ Không thể lấy thông tin tài chính: {e}")
        return None

def format_number(num, prefix='', suffix='', decimal=2):
    """Format số với đơn vị"""
    if num == 'N/A' or num is None:
        return 'N/A'

    try:
        num = float(num)
        if abs(num) >= 1e12:
            return f"{prefix}{num/1e12:.{decimal}f}T {suffix}".strip()
        elif abs(num) >= 1e9:
            return f"{prefix}{num/1e9:.{decimal}f}B {suffix}".strip()
        elif abs(num) >= 1e6:
            return f"{prefix}{num/1e6:.{decimal}f}M {suffix}".strip()
        elif abs(num) >= 1e3:
            return f"{prefix}{num/1e3:.{decimal}f}K {suffix}".strip()
        else:
            return f"{prefix}{num:.{decimal}f} {suffix}".strip()
    except:
        return str(num)

def print_fundamental_analysis(fundamentals, symbol):
    """In phân tích tài chính chi tiết"""
    if not fundamentals:
        print("⚠ Không có dữ liệu tài chính")
        return

    print("\n" + "="*70)
    print(f"  PHÂN TÍCH TÀI CHÍNH CƠ BẢN - {symbol}")
    print("="*70)

    # Thông tin công ty
    print("\n📋 THÔNG TIN CÔNG TY")
    print("-" * 70)
    print(f"Tên công ty:     {fundamentals['company_name']}")
    print(f"Ngành:           {fundamentals['sector']}")
    print(f"Lĩnh vực:        {fundamentals['industry']}")
    print(f"Nhân viên:       {format_number(fundamentals['employees'], suffix='người')}")

    # Giá và vốn hóa
    print("\n💰 GIÁ VÀ VỐN HÓA")
    print("-" * 70)
    current_price = fundamentals['current_price']
    print(f"Giá hiện tại:    {format_number(current_price, suffix='VND')}")
    print(f"Vốn hóa:         {format_number(fundamentals['market_cap'], suffix='VND')}")
    print(f"EV:              {format_number(fundamentals['enterprise_value'], suffix='VND')}")

    # So sánh với MA
    ma_50 = fundamentals['50d_avg']
    ma_200 = fundamentals['200d_avg']
    if current_price != 'N/A' and ma_50 != 'N/A':
        diff_50 = ((current_price - ma_50) / ma_50) * 100
        print(f"So với MA50:     {diff_50:+.2f}% ({format_number(ma_50, suffix='VND')})")
    if current_price != 'N/A' and ma_200 != 'N/A':
        diff_200 = ((current_price - ma_200) / ma_200) * 100
        print(f"So với MA200:    {diff_200:+.2f}% ({format_number(ma_200, suffix='VND')})")

    print(f"52W High:        {format_number(fundamentals['52w_high'], suffix='VND')}")
    print(f"52W Low:         {format_number(fundamentals['52w_low'], suffix='VND')}")

    # Chỉ số định giá
    print("\n📊 CHỈ SỐ ĐỊNH GIÁ")
    print("-" * 70)
    pe = fundamentals['pe_ratio']
    pb = fundamentals['pb_ratio']
    ps = fundamentals['ps_ratio']

    print(f"P/E Ratio:       {format_number(pe, decimal=2)}", end="")
    if pe != 'N/A' and pe < 15:
        print(" ✓ (Hấp dẫn)")
    elif pe != 'N/A' and pe > 25:
        print(" ⚠ (Cao)")
    else:
        print(" (Trung bình)")

    print(f"P/B Ratio:       {format_number(pb, decimal=2)}", end="")
    if pb != 'N/A' and pb < 1.5:
        print(" ✓ (Tốt)")
    elif pb != 'N/A' and pb > 3:
        print(" ⚠ (Cao)")
    else:
        print(" (Trung bình)")

    print(f"P/S Ratio:       {format_number(ps, decimal=2)}")
    print(f"PEG Ratio:       {format_number(fundamentals['peg_ratio'], decimal=2)}")

    # Chỉ số sinh lời
    print("\n💹 CHỈ SỐ SINH LỜI")
    print("-" * 70)
    profit_margin = fundamentals['profit_margin']
    roe = fundamentals['roe']
    roa = fundamentals['roa']

    if profit_margin != 'N/A':
        print(f"Profit Margin:   {profit_margin*100:.2f}%", end="")
        if profit_margin > 0.15:
            print(" ✓ (Tốt)")
        elif profit_margin < 0.05:
            print(" ⚠ (Thấp)")
        else:
            print()
    else:
        print(f"Profit Margin:   N/A")

    if fundamentals['operating_margin'] != 'N/A':
        print(f"Operating Margin: {fundamentals['operating_margin']*100:.2f}%")

    if roe != 'N/A':
        print(f"ROE:             {roe*100:.2f}%", end="")
        if roe > 0.15:
            print(" ✓ (Xuất sắc)")
        elif roe < 0.08:
            print(" ⚠ (Thấp)")
        else:
            print()
    else:
        print(f"ROE:             N/A")

    if roa != 'N/A':
        print(f"ROA:             {roa*100:.2f}%")

    # Tăng trưởng
    print("\n📈 TĂNG TRƯỞNG")
    print("-" * 70)
    rev_growth = fundamentals['revenue_growth']
    earn_growth = fundamentals['earnings_growth']

    if rev_growth != 'N/A':
        print(f"Tăng trưởng DT:  {rev_growth*100:+.2f}%")
    if earn_growth != 'N/A':
        print(f"Tăng trưởng LN:  {earn_growth*100:+.2f}%")

    # Cổ tức
    print("\n💵 CỔ TỨC")
    print("-" * 70)
    div_yield = fundamentals['dividend_yield']
    if div_yield != 'N/A' and div_yield > 0:
        print(f"Tỷ suất cổ tức:  {div_yield*100:.2f}%")
        print(f"Cổ tức/cổ phiếu: {format_number(fundamentals['dividend_rate'], suffix='VND')}")
        if fundamentals['payout_ratio'] != 'N/A':
            print(f"Tỷ lệ chi trả:   {fundamentals['payout_ratio']*100:.2f}%")
    else:
        print("Không có cổ tức hoặc chưa công bố")

    # Tình hình tài chính
    print("\n🏦 TÌNH HÌNH TÀI CHÍNH")
    print("-" * 70)
    total_cash = fundamentals['total_cash']
    total_debt = fundamentals['total_debt']

    print(f"Tiền mặt:        {format_number(total_cash, suffix='VND')}")
    print(f"Nợ:              {format_number(total_debt, suffix='VND')}")

    if total_cash != 'N/A' and total_debt != 'N/A' and total_cash > 0:
        net_debt = total_debt - total_cash
        print(f"Nợ ròng:         {format_number(net_debt, suffix='VND')}")

    current_ratio = fundamentals['current_ratio']
    if current_ratio != 'N/A':
        print(f"Current Ratio:   {current_ratio:.2f}", end="")
        if current_ratio > 2:
            print(" ✓ (Tốt)")
        elif current_ratio < 1:
            print(" ⚠ (Cảnh báo)")
        else:
            print()

    quick_ratio = fundamentals['quick_ratio']
    if quick_ratio != 'N/A':
        print(f"Quick Ratio:     {quick_ratio:.2f}")

    debt_to_equity = fundamentals['debt_to_equity']
    if debt_to_equity != 'N/A':
        print(f"Debt/Equity:     {debt_to_equity:.2f}", end="")
        if debt_to_equity < 0.5:
            print(" ✓ (Thấp)")
        elif debt_to_equity > 2:
            print(" ⚠ (Cao)")
        else:
            print()

    # Rủi ro và biến động
    print("\n⚡ RỦI RO & BIẾN ĐỘNG")
    print("-" * 70)
    beta = fundamentals['beta']
    if beta != 'N/A':
        print(f"Beta:            {beta:.2f}", end="")
        if beta < 1:
            print(" (Ít biến động hơn thị trường)")
        elif beta > 1.5:
            print(" (Biến động mạnh)")
        else:
            print(" (Biến động trung bình)")

    print(f"Khối lượng TB:   {format_number(fundamentals['avg_volume'], suffix='CP')}")

    # Khuyến nghị
    print("\n🎯 KHUYẾN NGHỊ")
    print("-" * 70)
    recommendation = fundamentals['recommendation']
    rec_map = {
        'buy': '🟢 MUA',
        'strong_buy': '🟢🟢 MUA MẠNH',
        'hold': '🟡 NẮM GIỮ',
        'sell': '🔴 BÁN',
        'strong_sell': '🔴🔴 BÁN MẠNH'
    }
    print(f"Khuyến nghị:     {rec_map.get(recommendation, recommendation.upper())}")

    target_price = fundamentals['target_price']
    if target_price != 'N/A' and current_price != 'N/A':
        potential = ((target_price - current_price) / current_price) * 100
        print(f"Giá mục tiêu:    {format_number(target_price, suffix='VND')} ({potential:+.2f}%)")

    print("="*70)

def analyze_financial_health(fundamentals):
    """Đánh giá tổng quan sức khỏe tài chính"""
    if not fundamentals:
        return None

    score = 0
    max_score = 0
    details = []

    # P/E Ratio (0-2 điểm)
    max_score += 2
    pe = fundamentals['pe_ratio']
    if pe != 'N/A':
        if 10 <= pe <= 20:
            score += 2
            details.append("✓ P/E hợp lý (10-20)")
        elif 5 <= pe < 10 or 20 < pe <= 25:
            score += 1
            details.append("○ P/E chấp nhận được")
        else:
            details.append("✗ P/E ngoài khoảng lý tưởng")

    # ROE (0-2 điểm)
    max_score += 2
    roe = fundamentals['roe']
    if roe != 'N/A':
        if roe >= 0.15:
            score += 2
            details.append("✓ ROE xuất sắc (≥15%)")
        elif roe >= 0.10:
            score += 1
            details.append("○ ROE tốt (≥10%)")
        else:
            details.append("✗ ROE thấp (<10%)")

    # Debt to Equity (0-2 điểm)
    max_score += 2
    d_to_e = fundamentals['debt_to_equity']
    if d_to_e != 'N/A':
        if d_to_e < 0.5:
            score += 2
            details.append("✓ Nợ rất thấp (<0.5)")
        elif d_to_e <= 1:
            score += 1
            details.append("○ Nợ hợp lý (≤1.0)")
        else:
            details.append("✗ Nợ cao (>1.0)")

    # Current Ratio (0-2 điểm)
    max_score += 2
    current_ratio = fundamentals['current_ratio']
    if current_ratio != 'N/A':
        if current_ratio >= 2:
            score += 2
            details.append("✓ Thanh khoản tốt (≥2.0)")
        elif current_ratio >= 1.5:
            score += 1
            details.append("○ Thanh khoản chấp nhận được")
        else:
            details.append("✗ Thanh khoản kém (<1.5)")

    # Profit Margin (0-2 điểm)
    max_score += 2
    profit_margin = fundamentals['profit_margin']
    if profit_margin != 'N/A':
        if profit_margin >= 0.15:
            score += 2
            details.append("✓ Lợi nhuận cao (≥15%)")
        elif profit_margin >= 0.08:
            score += 1
            details.append("○ Lợi nhuận ổn (≥8%)")
        else:
            details.append("✗ Lợi nhuận thấp (<8%)")

    if max_score > 0:
        health_score = (score / max_score) * 100

        if health_score >= 80:
            rating = "🟢 XUẤT SẮC"
        elif health_score >= 60:
            rating = "🟡 TỐT"
        elif health_score >= 40:
            rating = "🟠 TRUNG BÌNH"
        else:
            rating = "🔴 YẾU"

        return {
            'score': health_score,
            'rating': rating,
            'details': details
        }

    return None

### Dự báo:

In [12]:
# ============= PREDICTION =============
def predict_advanced(model, scaler, last_sequence, n_days, n_features):
    predictions = []
    current_seq = last_sequence.copy()

    for _ in range(n_days):
        # Convert NumPy array to TensorFlow tensor
        input_tensor = tf.convert_to_tensor(current_seq.reshape(1, current_seq.shape[0], current_seq.shape[1]), dtype=tf.float32)
        pred = model.predict(input_tensor, verbose=0)
        predictions.append(pred[0, 0])

        new_row = np.zeros((1, n_features))
        new_row[0, 0] = pred[0, 0]
        for i in range(1, n_features):
            new_row[0, i] = current_seq[-1, i]

        current_seq = np.vstack((current_seq[1:], new_row))

    dummy = np.zeros((len(predictions), n_features))
    dummy[:, 0] = predictions
    predictions = scaler.inverse_transform(dummy)[:, 0]

    return predictions

### Trực quan hóa:

In [13]:
# ============= VISUALIZATION =============
def plot_advanced_comparison(df, predictions_dict, risk_metrics, symbol, n_days):
    """Visualization với risk bands"""
    future_dates = [df['date'].iloc[-1] + timedelta(days=i+1) for i in range(n_days)]

    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        subplot_titles=('Dự báo Giá với Risk Bands', 'Volume', 'RSI'),
        vertical_spacing=0.08,
        row_heights=[0.5, 0.25, 0.25]
    )

    # Historical price
    fig.add_trace(
        go.Scatter(x=df['date'].tail(100), y=df['close'].tail(100),
                  name='Giá thực tế', line=dict(color='blue', width=2)),
        row=1, col=1
    )

    # Predictions - skip confidence bands
    colors = ['red', 'green', 'purple', 'orange', 'darkblue', 'pink', 'brown', 'cyan']
    dash_styles = ['dash', 'dot', 'dashdot', 'dash', 'solid', 'dot', 'dashdot', 'dash']

    pred_idx = 0
    for name, pred in predictions_dict.items():
        if name in ['GBM_lower', 'GBM_upper']:
            continue  # Skip these, will be added as confidence bands
        fig.add_trace(
            go.Scatter(x=future_dates, y=pred, name=name,
                      line=dict(color=colors[pred_idx % len(colors)],
                               dash=dash_styles[pred_idx % len(dash_styles)], width=2)),
            row=1, col=1
        )
        pred_idx += 1

    # GBM confidence bands
    if 'GBM_lower' in predictions_dict and 'GBM_upper' in predictions_dict:
        fig.add_trace(
            go.Scatter(x=future_dates, y=predictions_dict['GBM_upper'],
                      name='95% CI Upper', line=dict(width=0),
                      showlegend=False),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=future_dates, y=predictions_dict['GBM_lower'],
                      name='95% CI', fill='tonexty',
                      fillcolor='rgba(0,100,80,0.2)',
                      line=dict(width=0)),
            row=1, col=1
        )

    # Volume
    fig.add_trace(
        go.Bar(x=df['date'].tail(100), y=df['volume'].tail(100), name='Volume'),
        row=2, col=1
    )

    # RSI
    if 'RSI' in df.columns:
        fig.add_trace(
            go.Scatter(x=df['date'].tail(100), y=df['RSI'].tail(100),
                      name='RSI', line=dict(color='purple')),
            row=3, col=1
        )
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

    fig.update_layout(
        title=f'Dự báo {symbol} - {n_days} ngày (Ensemble Model với Risk Analysis)',
        xaxis_title='Ngày',
        height=900,
        hovermode='x unified'
    )

    return fig

## Chạy:

In [17]:
# ============= MAIN EXECUTION =============
if __name__ == "__main__":
    print("=" * 60)
    print("  HỆ THỐNG DỰ BÁO GIÁ CỔ PHIẾU NÂNG CAO")
    print("=" * 60)

    symbol_input = input("Nhập mã cổ phiếu (e.g., VCB): ") or "VCB"
    symbol = symbol_input.upper() + ".VN"
    n_days = int(input("Nhập số ngày dự đoán (1-30): ") or 7)
    years = int(input("Số năm dữ liệu (3-5): ") or 5)

    print("\n📊 Đang thu thập dữ liệu...")
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=365 * years)

    df = fetch_stock_data(symbol, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

    # Try multiple VN Index symbols
    vn_symbols = ['^VNINDEX', 'VNINDEX', '^VNI', 'VNI']
    vnindex_df = pd.DataFrame()

    for vn_sym in vn_symbols:
        print(f"🔍 Đang thử lấy VN Index với symbol: {vn_sym}")
        vnindex_df = fetch_stock_data(vn_sym, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
        if not vnindex_df.empty:
            break

    if vnindex_df.empty:
        print("⚠ Không lấy được VN Index, tiếp tục mà không có chỉ số này")

    if df.empty:
        print("❌ Không lấy được dữ liệu!")
    else:
        print(f"✓ Đã lấy {len(df)} ngày dữ liệu")

        # Display Fundamental Analysis
        print("\n🔍 Đang phân tích tài chính cơ bản...")
        fundamentals = get_company_fundamentals(symbol)
        if fundamentals:
            print_fundamental_analysis(fundamentals, symbol_input)
            health = analyze_financial_health(fundamentals)
            if health:
                print("\n📊 ĐÁNH GIÁ SỨC KHỎE TÀI CHÍNH")
                print("-" * 70)
                print(f"Điểm: {health['score']:.2f}/100")
                print(f"Xếp loại: {health['rating']}")
                print("Chi tiết:")
                for detail in health['details']:
                    print(f"- {detail}")
                print("-" * 70)
        else:
            print("⚠ Không thể thực hiện phân tích tài chính.")


        print("\n🔧 Đang tính toán chỉ báo kỹ thuật...")
        df = add_advanced_indicators(df, vnindex_df)

        print("\n🤖 Đang chuẩn bị dữ liệu cho mô hình...")
        X, y, scaler = prepare_data_advanced(df)
        n_features = X.shape[2]

        if len(X) < 100:
            print("❌ Dữ liệu không đủ!")
        else:
            # Define the custom layer before loading the model
            class AttentionLayer(Layer):
                """Custom Attention Layer"""
                def __init__(self, **kwargs):
                    super(AttentionLayer, self).__init__(**kwargs)

                def build(self, input_shape):
                    self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], input_shape[-1]),
                                            initializer='glorot_uniform', trainable=True)
                    self.b = self.add_weight(name='att_bias', shape=(input_shape[-1],),
                                            initializer='zeros', trainable=True)
                    super(AttentionLayer, self).build(input_shape)

                def call(self, x):
                    e = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
                    a = tf.nn.softmax(e, axis=1)
                    output = x * a
                    return tf.reduce_sum(output, axis=1)


            # Train or load model
            if os.path.exists(MODEL_PATH):
                print(f"\n📁 Đang tải mô hình từ {MODEL_PATH}...")
                model = load_model(MODEL_PATH, custom_objects={'AttentionLayer': AttentionLayer})
                # Calculate and display metrics for loaded model on validation set
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
                val_pred = model.predict(X_val, verbose=0)
                lstm_metrics = calculate_metrics(y_val, val_pred.flatten())
                print("\n📊 CHỈ SỐ ĐÁNH GIÁ MÔ HÌNH LSTM (trên tập Validation)")
                print("-" * 70)
                print(f"  MAE: {lstm_metrics['MAE']:.6f} | RMSE: {lstm_metrics['RMSE']:.6f}")
                print(f"  MAPE: {lstm_metrics['MAPE']:.2f}% | R²: {lstm_metrics['R2']:.4f}")
                print(f"  Directional Accuracy: {lstm_metrics['Directional_Accuracy']:.2f}%")
                print("-" * 70)

            else:
                print("\n🚀 Đang huấn luyện mô hình LSTM với Attention...")
                model, lstm_metrics = train_advanced_model(X, y, epochs=100, batch_size=32)
                model.save(MODEL_PATH)
                print("\n📊 CHỈ SỐ ĐÁNH GIÁ MÔ HÌNH LSTM SAU HUẤN LUYỆN")
                print("-" * 70)
                print(f"  MAE: {lstm_metrics['MAE']:.6f} | RMSE: {lstm_metrics['RMSE']:.6f}")
                print(f"  MAPE: {lstm_metrics['MAPE']:.2f}% | R²: {lstm_metrics['R2']:.4f}")
                print(f"  Directional Accuracy: {lstm_metrics['Directional_Accuracy']:.2f}%")
                print("-" * 70)


            print("\n🎯 Đang tạo dự báo với các mô hình...")

            # LSTM Prediction
            last_sequence = X[-1]
            lstm_pred = predict_advanced(model, scaler, last_sequence, n_days, n_features)

            # GBM Monte Carlo
            gbm_mean, gbm_lower, gbm_upper, gbm_sims = geometric_brownian_motion(df, n_days, n_sim=1000)

            # Mean-Reverting
            mr_pred = mean_reverting_forecast(df, n_days, n_sim=1000)

            # ARIMA (simple)
            try:
                arima_model = ARIMA(df['close'].tail(200), order=(5,1,0))
                arima_fit = arima_model.fit()
                arima_pred = arima_fit.forecast(steps=n_days)
            except:
                arima_pred = gbm_mean # Fallback if ARIMA fails

            # Ensemble
            ensemble_pred = ensemble_forecast(lstm_pred, gbm_mean, mr_pred, arima_pred)

            # Risk Metrics
            print("\n📈 Đang tính toán chỉ số rủi ro...")
            risk_metrics = calculate_risk_metrics(gbm_sims, df['close'].iloc[-1])

            print("\n" + "="*60)
            print("  CHỈ SỐ RỦI RO")
            print("="*60)
            print(f"Value at Risk (95%): {risk_metrics['VaR_95']*100:.2f}%")
            print(f"Conditional VaR (95%): {risk_metrics['CVaR_95']*100:.2f}%")
            print(f"Max Drawdown: {risk_metrics['Max_Drawdown']*100:.2f}%")
            print(f"Xác suất sinh lời: {risk_metrics['Prob_Profit']*100:.2f}%")

            # Plot
            predictions_dict = {
                'LSTM': lstm_pred,
                'GBM Monte Carlo': gbm_mean,
                'Mean-Reverting': mr_pred,
                'ARIMA': arima_pred,
                'Ensemble': ensemble_pred,
                'GBM_lower': gbm_lower,
                'GBM_upper': gbm_upper
            }

            fig = plot_advanced_comparison(df, predictions_dict, risk_metrics, symbol_input, n_days)
            fig.show()

            # Results table
            future_dates = [df['date'].iloc[-1] + timedelta(days=i+1) for i in range(n_days)]
            results_df = pd.DataFrame({
                'Ngày': future_dates,
                'LSTM': [f"{p:.0f}" for p in lstm_pred],
                'GBM': [f"{p:.0f}" for p in gbm_mean],
                'Mean-Rev': [f"{p:.0f}" for p in mr_pred],
                'ARIMA': [f"{p:.0f}" for p in arima_pred],
                'Ensemble': [f"{p:.0f}" for p in ensemble_pred]
            })

            print("\n" + "="*60)
            print("  KẾT QUẢ DỰ BÁO")
            print("="*60)
            print(results_df.to_string(index=False))

            print("\n✅ Hoàn thành!")

  HỆ THỐNG DỰ BÁO GIÁ CỔ PHIẾU NÂNG CAO
Nhập mã cổ phiếu (e.g., VCB): VNM
Nhập số ngày dự đoán (1-30): 5
Số năm dữ liệu (3-5): 3


ERROR:yfinance:$^VNINDEX: possibly delisted; no timezone found



📊 Đang thu thập dữ liệu...
✓ Lấy được 565 ngày dữ liệu cho VNM.VN
🔍 Đang thử lấy VN Index với symbol: ^VNINDEX
⚠ Thử lại 1/3...


ERROR:yfinance:$^VNINDEX: possibly delisted; no timezone found


⚠ Thử lại 2/3...


...

⚠ Thử lại 2/3...


ERROR:yfinance:$VNI: possibly delisted; no price data found  (1d 2022-11-06 -> 2025-11-05)


✗ Không thể lấy dữ liệu VNI: Không có dữ liệu
⚠ Không lấy được VN Index, tiếp tục mà không có chỉ số này
✓ Đã lấy 565 ngày dữ liệu

🔍 Đang phân tích tài chính cơ bản...

  PHÂN TÍCH TÀI CHÍNH CƠ BẢN - VNM

📋 THÔNG TIN CÔNG TY
----------------------------------------------------------------------
Tên công ty:     Vietnam Dairy Products Joint Stock Company
Ngành:           Consumer Defensive
Lĩnh vực:        Packaged Foods
Nhân viên:       9.79K người

💰 GIÁ VÀ VỐN HÓA
----------------------------------------------------------------------
Giá hiện tại:    58.00K VND
Vốn hóa:         121.22T VND
EV:              105.50T VND
So với MA50:     -4.22% (60.56K VND)
So với MA200:    -2.61% (59.55K VND)
52W High:        66.20K VND
52W Low:         51.40K VND

📊 CHỈ SỐ ĐỊNH GIÁ
----------------------------------------------------------------------
P/E Ratio:       15.62 (Trung bình)
P/B Ratio:       3.55 ⚠ (Cao)
P/S Ratio:       1.95
PEG Ratio:       N/A

💹 CHỈ SỐ SINH LỜI
-----------------------


📊 CHỈ SỐ ĐÁNH GIÁ MÔ HÌNH LSTM (trên tập Validation)
----------------------------------------------------------------------
  MAE: 0.129260 | RMSE: 0.154094
  MAPE: 27.25% | R²: -1.4736
  Directional Accuracy: 45.00%
----------------------------------------------------------------------

🎯 Đang tạo dự báo với các mô hình...

📈 Đang tính toán chỉ số rủi ro...

  CHỈ SỐ RỦI RO
Value at Risk (95%): -0.26%
Conditional VaR (95%): -0.34%
Max Drawdown: -0.50%
Xác suất sinh lời: 52.10%



  KẾT QUẢ DỰ BÁO
      Ngày  LSTM   GBM Mean-Rev ARIMA Ensemble
2025-11-05 58571 57300    57300 57336    57812
2025-11-06 58590 57302    57301 57312    57818
2025-11-07 58604 57302    57301 57318    57824
2025-11-08 58617 57301    57304 57323    57830
2025-11-09 58636 57304    57306 57321    57839

✅ Hoàn thành!
